In [1]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display
import io

# Check for GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Upload Targeted Station CSV
target_upload = widgets.FileUpload(accept='.csv', multiple=False)
display(widgets.Label(value="Upload Targeted Station CSV"), target_upload)

# Upload Multiple Selected Stations CSVs
selected_upload = widgets.FileUpload(accept='.csv', multiple=True)
display(widgets.Label(value="Upload Selected Stations CSVs"), selected_upload)

# Wait for user to upload files
while not target_upload.value or not selected_upload.value:
    pass  # Wait until files are uploaded

# Read targeted station CSV
target_df = pd.read_csv(io.BytesIO(list(target_upload.value.values())[0]['content']))

# Read and merge multiple selected stations CSVs
input_dfs = [pd.read_csv(io.BytesIO(file['content'])) for file in selected_upload.value.values()]
input_df = pd.concat(input_dfs, ignore_index=True)  # Combine into one DataFrame

# Sort data by date
target_df = target_df.sort_values(by="Date")
input_df = input_df.sort_values(by="Date")

# Normalize features
scaler = MinMaxScaler()
target_df.iloc[:, 1:] = scaler.fit_transform(target_df.iloc[:, 1:])  # Assuming features start from column 1
input_df.iloc[:, 1:] = scaler.transform(input_df.iloc[:, 1:])

# Convert to NumPy arrays
X = input_df.iloc[:, 1:].values  # Input features
Y = target_df.iloc[:, 1:].values  # Output features

# Function to create sequences
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)

# Define Optimized RNN Model
model = Sequential([
    LSTM(20, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),  # Reduced from 50 to 20
    LSTM(20, activation='relu'),  # Reduced from 50 to 20
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model (Optimized Training)
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)  # Reduced epochs from 50 to 10

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Convert predictions back to original scale
real_predictions = scaler.inverse_transform(predicted_weather)

# Display Predictions
print("Predicted Weather for Next 7 Days (Scaled Values):")
print(predicted_weather)

print("Predicted Weather for Next 7 Days (Real Values):")
print(real_predictions)

In [1]:
# Install required libraries (if not already installed)
!pip install tensorflow pandas numpy scikit-learn

# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os

# Check for GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Upload Targeted Station CSV (use file dialog in Jupyter)
from tkinter import Tk
from tkinter.filedialog import askopenfilename

print("Upload Targeted Station CSV")
Tk().withdraw()  # Prevents the root window from appearing
target_filename = askopenfilename(title="Select Targeted Station CSV")
target_df = pd.read_csv(target_filename)

# Upload Multiple Selected Stations CSVs
print("Upload Selected Stations CSVs")
selected_filenames = askopenfilename(title="Select Selected Stations CSV", multiple=True)
selected_dfs = [pd.read_csv(filename) for filename in selected_filenames]

# Sort data by date (assuming 'Date' column exists)
if 'Date' in target_df.columns:
    target_df = target_df.sort_values(by="Date")
else:
    target_df = target_df.sort_values(by="DATE")  # Try 'DATE' if 'Date' is not found

# Normalize features (excluding Date and TIME(UTC) columns)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# 🔸 Select only numerical columns for scaling
numerical_cols_input = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar
numerical_cols_target = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar

X = scaler_x.fit_transform(target_df[numerical_cols_input].values)  # Input features
Y = scaler_y.fit_transform(target_df[numerical_cols_target].values)  # Output features

# Function to create sequences for RNN
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    num_sequences = min(len(X), len(Y)) - time_steps 
    for i in range(num_sequences):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)

# Define Optimized RNN Model
model = Sequential([
    LSTM(32, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    LSTM(32, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Convert predictions back to original scale
real_predictions = scaler_y.inverse_transform(predicted_weather)

# Display Predictions
print("\nPredicted Weather for Next 7 Days (Scaled Values):")
print(predicted_weather)

print("\nPredicted Weather for Next 7 Days (Real Values):")
print(real_predictions)

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Upload Targeted Station CSV


2025-02-19 11:04:59.001 Python[2402:18130] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 11:04:59.391 Python[2402:18130] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


Upload Selected Stations CSVs


/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
2025-02-19 11:05:11.961482: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-02-19 11:05:11.961655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-02-1

Epoch 1/10


2025-02-19 11:05:12.678840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 139ms/step - loss: nan
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 134ms/step - loss: nan
Epoch 3/10
18/92 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - loss: nan

: 

In [2]:
# Check for NaN values in the dataset
print("Target Data NaN values:", target_df.isna().sum())
print("Input Data NaN values:", input_df.isna().sum())

# Handle NaN values (e.g., fill with mean or drop rows)
target_df = target_df.fillna(target_df.mean())
input_df = input_df.fillna(input_df.mean())

Target Data NaN values: DATE                         0
TIME(UTC)                    0
RAIN_HOURLY(mm)           2771
RAIN_DAILY(mm)               0
TEMP(C)                      0
TEMP_MIN(C)                  0
TEMP_MAX(C)                  0
TEMP_MIN_MAX_DAY(C)       2709
DEW_POINT(C)              2771
RH(%)                        0
RH_MIN_MAX_DAY(%)         2771
WIND_DIR_10m(deg)         2771
WIND_SPEED_10m(kt)        2771
WIND_MAX/GUST_10m(kt)     2771
WIND_DIR_3m(deg)          2771
WIND_SPEED_3m(kt)         2771
WIND_MAX/GUST_3m(kt)      2771
SLP(hPa)                  2771
MSLP(hPa)                 2771
SUNSHINE(HH:MM)           2771
SOIL_TEMP_10cm(C)         2771
SOIL_MOIS_10cm(m3/m3)     2771
SOIL_TEMP_30cm(C)         2771
SOIL_MOIS_30cm(m3/m3)     2771
SOIL_TEMP_70cm(C)         2771
SOIL_MOIS_70cm(m3/m3)     2771
SOIL_TEMP_100cm(C)        2771
SOIL_MOIS_100cm(m3/m3)    2771
GLOBAL_RADIATION(v/m2)       0
PAR(micro-mole/m2s)       2771
Unnamed: 30               2771
dtype: int64


NameError: name 'input_df' is not defined

In [3]:
print("X_train Shape:", X_train.shape)

X_train Shape: (2764, 7, 28)


In [4]:
print("X_train Sample:", X_train[0])
print("Y_train Sample:", Y_train[0])

X_train Sample: [[0.         0.215      0.2345679  0.55900621        nan        nan
  0.67058824        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan 0.                nan        nan]
 [0.         0.395      0.49382716 0.60869565        nan        nan
  0.68235294        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan 0.                nan        nan]
 [0.         0.395      0.49382716 0.60869565        nan        nan
  0.69411765        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan 0.                nan        nan]
 [0.         0.4        0.5

In [5]:
# Check for NaN values in the dataset
print("Target Data NaN values:", target_df.isna().sum())
print("Input Data NaN values:", input_df.isna().sum())

# Handle NaN values (e.g., fill with mean or drop rows)
target_df = target_df.fillna(target_df.mean())
input_df = input_df.fillna(input_df.mean())

# Normalize features (excluding Date and TIME(UTC) columns)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# Select only numerical columns for scaling
numerical_cols_input = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar
numerical_cols_target = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar

X = scaler_x.fit_transform(target_df[numerical_cols_input].values)  # Input features
Y = scaler_y.fit_transform(target_df[numerical_cols_target].values)  # Output features

# Check the scaling output
print("Sample X after scaling:", X[0])
print("Sample Y after scaling:", Y[0])

# Function to create sequences for RNN
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    num_sequences = min(len(X), len(Y)) - time_steps 
    for i in range(num_sequences):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)

# Check the shape of X_train and Y_train
print("X_train Shape:", X_train.shape)
print("Y_train Shape:", Y_train.shape)

# Define Optimized RNN Model
model = Sequential([
    LSTM(32, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    LSTM(32, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Convert predictions back to original scale
real_predictions = scaler_y.inverse_transform(predicted_weather)

# Display Predictions
print("\nPredicted Weather for Next 7 Days (Scaled Values):")
print(predicted_weather)

print("\nPredicted Weather for Next 7 Days (Real Values):")
print(real_predictions)

Target Data NaN values: DATE                         0
TIME(UTC)                    0
RAIN_HOURLY(mm)           2771
RAIN_DAILY(mm)               0
TEMP(C)                      0
TEMP_MIN(C)                  0
TEMP_MAX(C)                  0
TEMP_MIN_MAX_DAY(C)       2709
DEW_POINT(C)              2771
RH(%)                        0
RH_MIN_MAX_DAY(%)         2771
WIND_DIR_10m(deg)         2771
WIND_SPEED_10m(kt)        2771
WIND_MAX/GUST_10m(kt)     2771
WIND_DIR_3m(deg)          2771
WIND_SPEED_3m(kt)         2771
WIND_MAX/GUST_3m(kt)      2771
SLP(hPa)                  2771
MSLP(hPa)                 2771
SUNSHINE(HH:MM)           2771
SOIL_TEMP_10cm(C)         2771
SOIL_MOIS_10cm(m3/m3)     2771
SOIL_TEMP_30cm(C)         2771
SOIL_MOIS_30cm(m3/m3)     2771
SOIL_TEMP_70cm(C)         2771
SOIL_MOIS_70cm(m3/m3)     2771
SOIL_TEMP_100cm(C)        2771
SOIL_MOIS_100cm(m3/m3)    2771
GLOBAL_RADIATION(v/m2)       0
PAR(micro-mole/m2s)       2771
Unnamed: 30               2771
dtype: int64


NameError: name 'input_df' is not defined

In [6]:
# Install required libraries (if not already installed)
!pip install tensorflow pandas numpy scikit-learn

# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os

# Check for GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Upload Targeted Station CSV (use file dialog in Jupyter)
from tkinter import Tk
from tkinter.filedialog import askopenfilename

print("Upload Targeted Station CSV")
Tk().withdraw()  # Prevents the root window from appearing
target_filename = askopenfilename(title="Select Targeted Station CSV")
target_df = pd.read_csv(target_filename)

# Sort data by date (assuming 'Date' column exists)
if 'Date' in target_df.columns:
    target_df = target_df.sort_values(by="Date")
else:
    target_df = target_df.sort_values(by="DATE")  # Try 'DATE' if 'Date' is not found

# Check for NaN values in the dataset
print("Target Data NaN values:", target_df.isna().sum())

# Handle NaN values (e.g., fill with mean or drop rows)
target_df = target_df.fillna(target_df.mean())

# Normalize features (excluding Date and TIME(UTC) columns)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# Select only numerical columns for scaling
numerical_cols = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar

X = scaler_x.fit_transform(target_df[numerical_cols].values)  # Input features
Y = scaler_y.fit_transform(target_df[numerical_cols].values)  # Output features

# Check the scaling output
print("Sample X after scaling:", X[0])
print("Sample Y after scaling:", Y[0])

# Function to create sequences for RNN
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    num_sequences = min(len(X), len(Y)) - time_steps 
    for i in range(num_sequences):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)

# Check the shape of X_train and Y_train
print("X_train Shape:", X_train.shape)
print("Y_train Shape:", Y_train.shape)

# Define Optimized RNN Model
model = Sequential([
    LSTM(32, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    LSTM(32, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Convert predictions back to original scale
real_predictions = scaler_y.inverse_transform(predicted_weather)

# Display Predictions
print("\nPredicted Weather for Next 7 Days (Scaled Values):")
print(predicted_weather)

print("\nPredicted Weather for Next 7 Days (Real Values):")
print(real_predictions)

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Upload Targeted Station CSV


2025-02-18 18:45:18.334 Python[3183:61377] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Target Data NaN values: DATE                         0
TIME(UTC)                    0
RAIN_HOURLY(mm)           2926
RAIN_DAILY(mm)               0
TEMP(C)                      0
TEMP_MIN(C)                  0
TEMP_MAX(C)                  0
TEMP_MIN_MAX_DAY(C)       2864
DEW_POINT(C)               278
RH(%)                      273
RH_MIN_MAX_DAY(%)         2926
WIND_DIR_10m(deg)         2926
WIND_SPEED_10m(kt)        2926
WIND_MAX/GUST_10m(kt)     2926
WIND_DIR_3m(deg)          2926
WIND_SPEED_3m(kt)         2926
WIND_MAX/GUST_3m(kt)      2926
SLP(hPa)                     1
MSLP(hPa)                    1
SUNSHINE(HH:MM)           2926
SOIL_TEMP_10cm(C)         2926
SOIL_MOIS_10cm(m3/m3)     2926
SOIL_TEMP_30cm(C)         2926
SOIL_MOIS_30cm(m3/m3)     2926
SOIL_TEMP_70cm(C)         2926
SOIL_MOIS_70cm(m3/m3)     2926
SOIL_TEMP_100cm(C)        2926
SOIL_MOIS_100cm(m3/m3)    2926
GLOBAL_RADIATION(v/m2)       0
PAR(micro-mole/m2s)       2926
Unnamed: 30               2926
dtype: int64


TypeError: Could not convert ['2024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-012024-12-0

In [2]:
# Install required libraries (if not already installed)
# !pip install tensorflow pandas numpy scikit-learn

# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os

# Check for GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Upload Targeted Station CSV (use file dialog in Jupyter)
from tkinter import Tk
from tkinter.filedialog import askopenfilename

print("Upload Targeted Station CSV")
Tk().withdraw()  # Prevents the root window from appearing
target_filename = askopenfilename(title="Select Targeted Station CSV")
target_df = pd.read_csv(target_filename)

# Upload Multiple Selected Stations CSVs
print("Upload Selected Stations CSVs")
selected_filenames = askopenfilename(title="Select Selected Stations CSV", multiple=True)
selected_dfs = [pd.read_csv(filename) for filename in selected_filenames]

# Sort data by date (assuming 'Date' column exists)
if 'Date' in target_df.columns:
    target_df = target_df.sort_values(by="Date")
else:
    target_df = target_df.sort_values(by="DATE")  # Try 'DATE' if 'Date' is not found

# Normalize features (excluding Date and TIME(UTC) columns)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# 🔸 Select only numerical columns for scaling
numerical_cols_input = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar
numerical_cols_target = target_df.select_dtypes(include=np.number).columns[1:]  # Exclude 'DATE' or similar

X = scaler_x.fit_transform(target_df[numerical_cols_input].values)  # Input features
Y = scaler_y.fit_transform(target_df[numerical_cols_target].values)  # Output features

# Function to create sequences for RNN
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    num_sequences = min(len(X), len(Y)) - time_steps 
    for i in range(num_sequences):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)

# Define Optimized RNN Model
model = Sequential([
    LSTM(32, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    LSTM(32, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Convert predictions back to original scale
real_predictions = scaler_y.inverse_transform(predicted_weather)

# Display Predictions
print("\nPredicted Weather for Next 7 Days (Scaled Values):")
print(predicted_weather)

print("\nPredicted Weather for Next 7 Days (Real Values):")
print(real_predictions)

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Upload Targeted Station CSV


2025-02-19 11:11:36.231 Python[3149:22295] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Upload Selected Stations CSVs
Epoch 1/10


/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/rohitlabade/Desktop/IMD-Project/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, 

92/92 ━━━━━━━━━━━━━━━━━━━━ 15s 145ms/step - loss: nan
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 143ms/step - loss: nan
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 143ms/step - loss: nan
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 142ms/step - loss: nan
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 143ms/step - loss: nan
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 147ms/step - loss: nan
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 151ms/step - loss: nan
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 156ms/step - loss: nan
Epoch 9/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - loss: nan
Epoch 10/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 15s 158ms/step - loss: nan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step

Predicted Weather for Next 7 Days (Scaled Values):
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan]]

Predicted Weather for Next 7 Days (Real Values):
[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan n